In [1]:
y_pred = [[0.5000368],[0.5000373],
[0.5000471],
[0.50004],
[0.50005],
[0.50005245],
[0.50005275],
[0.5000449],
[0.5000544],
[0.50005734],
[0.5000564],
[0.5000378],
[0.5000452],
[0.5000574],
[0.50004077],
[0.50004494]]

y_true = [[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.]]

In [2]:
def get_model():
    from tensorflow.keras.models import Sequential
    from keras import initializers
    from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [3]:
import numpy as np
import os
def fullpath(path, files):
    return  [(lambda x: path + x)(x) for x in files]
val_dir = "../chest_xray/chest_xray/val"
filesPneumo = fullpath(val_dir+'/PNEUMONIA/',os.listdir(os.path.join(val_dir, 'PNEUMONIA')))
del filesPneumo[0]
labelsPneumo = np.zeros(len(filesPneumo))
filesNormal = fullpath(val_dir+'/NORMAL/',os.listdir(os.path.join(val_dir, 'NORMAL')))
del filesNormal[0]
labelsNormal = np.ones(len(filesNormal))
fileData = filesPneumo + filesNormal
fileLabels = np.concatenate((labelsPneumo,labelsNormal),axis=0).astype(np.float32)

In [4]:
import tensorflow as tf
import numpy as np

model = get_model()

last_layer = model.layers[-1]

last_layer_weigths = last_layer.trainable_weights
kernel_W = last_layer_weigths[0]
bias_W = last_layer_weigths[1]

Using TensorFlow backend.


In [20]:
from PIL import Image
from mxnet import autograd, np as np2, npx
npx.set_np()
x = Image.open(filesNormal[0])
x = x.resize([320,320])
x = np.asarray(x).reshape(320*320*1)

In [21]:
x2 = np2.array(x)
x2.attach_grad()
x2.grad

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [22]:
x = x.reshape(1,320,320,1)
#x = x.reshape(-1,1)

In [23]:
dataset = tf.data.Dataset.from_tensor_slices(np.asarray(x))

In [27]:
with autograd.record():
    y = model.predict(x)
y = np2.array(y)

In [29]:
y.backward(retain_graph=True)

MXNetError: Traceback (most recent call last):
  File "src/imperative/imperative.cc", line 295
MXNetError: Check failed: !AGInfo: :IsNone(*i): Cannot differentiate node because it is not in a computational graph. You need to set is_recording to true or use autograd.record() to save computational graphs for backward. If you want to differentiate the same graph twice, you need to pass retain_graph=True to backward.